## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Ponta Do Sol,PT,72.23,clear sky,32.6667,-17.1000,Hotel do Campo
1,1,La Ronge,CA,82.44,overcast clouds,55.1001,-105.2842,Northland Motor Hotel
2,2,Yinchuan,CN,73.71,light rain,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
3,3,Kavieng,PG,83.48,light rain,-2.5744,150.7967,Nusa Island Retreat
4,4,Mecca,SA,85.64,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"


In [19]:
subset_df = vacation_df.loc[(vacation_df["Country"] == "US")]
subset_df.head()


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,13,Kapaa,US,85.98,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,15,Hilo,US,78.24,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
20,20,Flagstaff,US,89.89,clear sky,35.1981,-111.6513,Drury Inn & Suites Flagstaff
25,25,Havelock,US,80.55,broken clouds,34.8791,-76.9013,Sherwood Motel
36,36,Makakilo City,US,88.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [21]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [22]:
# 4a. Add a marker layer for each city to the map.

# 4b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

#### Four DataFrames are created, one for each city on the itinerary

In [23]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"]=="Saint Anthony"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Saint Anthony"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Lincoln"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Bentonville"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Bud"]  

#### The latitude and longitude pairs for each of the four cities are retrieved

In [24]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')

stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3: {stop3}')


Start: (45.0205, -93.218), End: (45.0205, -93.218)
Stop 1: (40.8, -96.667), Stop 2: (36.3728, -94.2088), Stop 3: (39.447, -86.1758)


#### A directions layer map between the cities and the travel map is created

In [25]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2,stop3],
        travel_mode='DRIVING')

fig.add_layer(city_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

#### A DataFrame that contains the four cities on the itinerary is created

In [33]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,88,Saint Anthony,US,84.15,broken clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
1,191,Lincoln,US,82.65,clear sky,40.8000,-96.6670,Courtyard by Marriott Lincoln Downtown/Haymarket
2,160,Bentonville,US,84.11,clear sky,36.3728,-94.2088,Hilton Garden Inn Bentonville Rogers
3,257,Bud,US,77.94,clear sky,39.4470,-86.1758,Ted McQuinn Trust


#### Format Max temp to one decimal place

In [34]:
itinerary_df["Max Temp"] = itinerary_df["Max Temp"].map("{:,.1f}".format)
# itinerary_df.head()

In [35]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

#### A marker layer map with a pop-up marker for the cities on the itinerary is created

In [36]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))